In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
train_transaction = pd.read_csv('../input/train_transaction.csv')
test_transaction = pd.read_csv('../input/test_transaction.csv')
train_identity = pd.read_csv('../input/train_identity.csv')
test_identity = pd.read_csv('../input/test_identity.csv')

In [3]:
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0,0,5069957,440.95,W,13809,583.0,150.0,visa,226.0,credit,204.0,87.0,852.0,NaN,yahoo.com,NaN,2.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,39.0,T,F,F,NaN,NaN,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,440.950012,440.950012,440.950012,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,440.950012,440.950012,440.950012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,14773564,460.00,W,12695,490.0,150.0,visa,226.0,debit,325.0,87.0,10.0,NaN,hotmail.com,NaN,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,40.0,2.0,102.0,102.0,3.0,424.0,3.0,NaN,NaN,NaN,NaN,190.0,84.0,NaN,NaN,NaN,102.0,T,T,T,M0,F,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,

In [4]:
test_transaction.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0,242834,25.000,H,9803,583.0,150.0,visa,226.0,credit,269.0,87.0,NaN,NaN,yahoo.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,7.791666,0.791666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,15.0,136.0,0.0,0.0,0.0,0.0,1822.0,47.0,64.0,0.0,0.0,0.0,0.0,0.0,0.0,15951.990234,198853.765625,0.0,0.0,0.0,915.0,5975.0,1320.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,15123000,117.000,W,7919,194.0,150.0,mastercard,166.0,debit,181.0,87.0,483.0,NaN,gmail.com,NaN,1.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,3.0,0.0,4.0,1.0,63.0,35.0,1.0,4.0,4.0,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,T,T,T,M0,F,F,F,F,F,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,

In [5]:
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,0,0.0,82050.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,NaN,529.0,178.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10.12,firefox 56.0,24.0,1920x1080,match_status:2,T,F,T,T,desktop,rv:56.0
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-5.0,183599.0,NaN,NaN,6.0,-11.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,15.0,352.0,280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 10.3.3,mobile safari generic,32.0,1136x640,match_status:2,T,F,T,F,mobile,iOS Device
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472432 entries, 0 to 472431
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.4+ GB


In [8]:
test_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118108 entries, 0 to 118107
Columns: 393 entries, TransactionID to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 354.1+ MB


In [9]:
train_identity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472432 entries, 0 to 472431
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  472432 non-null  int64  
 1   id_01          115525 non-null  float64
 2   id_02          112851 non-null  float64
 3   id_03          53091 non-null   float64
 4   id_04          53091 non-null   float64
 5   id_05          109618 non-null  float64
 6   id_06          109618 non-null  float64
 7   id_07          4115 non-null    float64
 8   id_08          4115 non-null    float64
 9   id_09          60010 non-null   float64
 10  id_10          60010 non-null   float64
 11  id_11          112940 non-null  float64
 12  id_12          115525 non-null  object 
 13  id_13          101926 non-null  float64
 14  id_14          64172 non-null   float64
 15  id_15          112946 non-null  object 
 16  id_16          103635 non-null  object 
 17  id_17          111626 non-nul

In [10]:
test_identity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118108 entries, 0 to 118107
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  118108 non-null  int64  
 1   id_01          28708 non-null   float64
 2   id_02          28021 non-null   float64
 3   id_03          13233 non-null   float64
 4   id_04          13233 non-null   float64
 5   id_05          27247 non-null   float64
 6   id_06          27247 non-null   float64
 7   id_07          1040 non-null    float64
 8   id_08          1040 non-null    float64
 9   id_09          14916 non-null   float64
 10  id_10          14916 non-null   float64
 11  id_11          28038 non-null   float64
 12  id_12          28708 non-null   object 
 13  id_13          25394 non-null   float64
 14  id_14          15872 non-null   float64
 15  id_15          28039 non-null   object 
 16  id_16          25705 non-null   object 
 17  id_17          27743 non-null

In [11]:
train_transaction.shape, train_identity.shape

((472432, 394), (472432, 41))

In [12]:
test_transaction.shape, test_identity.shape

((118108, 393), (118108, 41))

In [13]:
len(set(train_transaction['TransactionID']) & set(train_identity['TransactionID']))

472432

In [14]:
len(set(test_transaction['TransactionID']) & set(test_identity['TransactionID']))

118108

## train_identity

In [15]:
train_identity['DeviceType'].unique()

array([nan, 'desktop', 'mobile'], dtype=object)

In [16]:
train_identity['DeviceInfo'].unique(), train_identity['DeviceInfo'].nunique()

(array([nan, 'Windows', 'LG-K410 Build/LRX22G', ..., 'ASUS_Z00UD',
        'SAMSUNG-SM-T677A', 'SAMSUNG SM-J111M Build/LMY47V'], dtype=object),
 1687)

In [17]:
for i in ['id_35', 'id_36', 'id_37', 'id_38']:
    print(i)
    print(train_identity[i].unique())

id_35
[nan 'F' 'T']
id_36
[nan 'F' 'T']
id_37
[nan 'T' 'F']
id_38
[nan 'F' 'T']


In [18]:
for i in ['id_12', 'id_15', 'id_16', 'id_28', 'id_29']:
    print(i)
    print(train_identity[i].unique())

id_12
[nan 'NotFound' 'Found']
id_15
[nan 'New' 'Found' 'Unknown']
id_16
[nan 'NotFound' 'Found']
id_28
[nan 'New' 'Found']
id_29
[nan 'NotFound' 'Found']


In [19]:
train_identity['id_23'].unique(), train_identity['id_23'].nunique()

(array([nan, 'IP_PROXY:ANONYMOUS', 'IP_PROXY:TRANSPARENT',
        'IP_PROXY:HIDDEN'], dtype=object),
 3)

In [20]:
train_identity['id_27'].unique(), train_identity['id_27'].nunique()

(array([nan, 'Found', 'NotFound'], dtype=object), 2)

In [21]:
train_identity['id_30'].unique(), train_identity['id_30'].nunique()

(array([nan, 'Windows 7', 'Windows 10', 'Android 6.0.1', 'Windows 8.1',
        'Mac OS X 10_12_6', 'iOS 11.0.3', 'Mac OS X 10_13_2', 'Windows 8',
        'Android 7.0', 'Android', 'iOS 11.1.2', 'Mac OS X 10.13',
        'Mac OS X 10_11_6', 'iOS 11.2.1', 'Mac OS X 10_9_5', 'Linux',
        'Android 7.1.1', 'iOS 11.3.0', 'iOS 10.3.3', 'Mac OS X 10_13_1',
        'Mac OS X 10.10', 'Mac OS X 10_13_4', 'Mac OS X 10_13_3',
        'iOS 11.1.0', 'iOS 10.2.1', 'iOS 9.3.5', 'iOS 11.2.5',
        'iOS 10.2.0', 'iOS 11.1.1', 'iOS 10.3.2', 'Mac OS X 10_10_5',
        'Mac OS X 10_11_4', 'iOS 11.2.2', 'iOS 11.2.0', 'Mac OS X 10.11',
        'Mac OS X 10_8_5', 'iOS 10.0.2', 'iOS 10.3.1', 'Android 8.1.0',
        'Mac OS X 10.12', 'Mac OS X 10_12_4', 'Mac', 'iOS 11.2.6',
        'Android 5.1.1', 'Android 8.0.0', 'Mac OS X 10_12_5',
        'Windows Vista', 'Mac OS X 10_12_3', 'Mac OS X 10_11_5',
        'iOS 11.3.1', 'iOS 11.0.2', 'iOS 11.0.0', 'Mac OS X 10.9', 'iOS',
        'Android 5.0', 'Windows

In [22]:
# id_31
train_identity['id_31'].unique(), train_identity['id_31'].nunique()

(array([nan, 'chrome 65.0', 'chrome 66.0', 'chrome 63.0',
        'chrome 62.0 for android', 'chrome 62.0', 'chrome generic',
        'chrome 56.0 for android', 'chrome generic for android',
        'mobile safari generic', 'ie 11.0 for desktop',
        'chrome 63.0 for android', 'mobile safari 11.0', 'safari generic',
        'chrome 64.0', 'firefox 57.0', 'chrome 66.0 for android',
        'firefox 59.0', 'chrome 64.0 for android', 'ie 11.0 for tablet',
        'edge 16.0', 'edge 15.0', 'firefox 58.0', 'safari 11.0',
        'firefox 52.0', 'Generic/Android 7.0', 'chrome 61.0 for android',
        'mobile safari 10.0', 'samsung browser 6.2', 'mobile safari 9.0',
        'chrome 49.0', 'firefox 48.0', 'chrome 49.0 for android',
        'chrome 60.0 for android', 'chrome 55.0 for android',
        'chrome 65.0 for android', 'edge 13.0', 'chrome 55.0', 'other',
        'chrome 63.0 for ios', 'edge 14.0', 'firefox', 'Generic/Android',
        'Mozilla/Firefox', 'android webview 4.0', 'c

In [23]:
# id_33
train_identity['id_33'].unique(), train_identity['id_33'].nunique()

(array([nan, '1600x900', '1920x1080', '1280x1024', '2562x1442', '1366x768',
        '2880x1800', '1334x750', '1024x768', '1136x640', '1680x1050',
        '2001x1125', '1280x800', '1440x900', '2560x1440', '2732x2048',
        '1536x960', '2208x1242', '1280x720', '2436x1125', '2048x1536',
        '1599x900', '1920x1200', '2220x1080', '2160x1440', '2736x1824',
        '855x480', '3360x2100', '5120x2880', '2672x1440', '4096x2304',
        '1366x767', '3200x1800', '2560x1600', '1280x960', '3000x2000',
        '1360x768', '1024x600', '801x480', '1365x768', '1152x864',
        '1281x801', '2224x1668', '3840x2160', '1280x1025', '1279x1023',
        '960x540', '1848x1155', '1600x1200', '2220x1081', '1921x1081',
        '0x0', '2560x1080', '2049x1152', '1536x864', '2961x1442',
        '2400x1350', '3840x1080', '1919x1080', '3840x2400', '3440x1440',
        '2961x1440', '1918x1080', '2960x1440', '1502x844', '1680x1051',
        '2400x1600', '1280x768', '2735x1825', '2048x1280', '2737x1825',
     

In [24]:
# id_34
train_identity['id_34'].unique(), train_identity['id_34'].nunique()

(array([nan, 'match_status:2', 'match_status:1', 'match_status:0',
        'match_status:-1'], dtype=object),
 4)

## train_transaction

In [25]:
# ProductCD
train_transaction['ProductCD'].unique(), train_transaction['ProductCD'].nunique(),

(array(['W', 'C', 'R', 'H', 'S'], dtype=object), 5)

In [27]:
for i in ['card{}'.format(x) for x in range(1, 7)]:
    print(i)
    print(train_transaction[i].unique(), train_transaction[i].nunique())

card1
[13809 12695 18132 ... 14508 18290 14093] 12750
card2
[583. 490. 567. 111. 553. 204. 555. 500. 455. 201. 595. 514. 321. 264.
 215. 327. 194. 225. 543. 360. 174. 408. 513. 158. 383.  nan 545. 144.
 170. 494. 453. 122. 103. 445. 257. 558. 417. 452. 540. 226. 361. 399.
 470. 177. 418. 100. 206. 298. 250. 598. 393. 303. 476. 268. 428. 272.
 254. 593. 297. 277. 478. 355. 154. 247. 562. 532. 512. 310. 181. 525.
 481. 372. 510. 375. 390. 480. 302. 182. 209. 496. 324. 579. 234. 517.
 242. 411. 584. 106. 528. 554. 300. 273. 291. 308. 104. 205. 507. 312.
 258. 130. 239. 269. 296. 338. 304. 161. 459. 142. 325. 396. 585. 148.
 150. 202. 203. 143. 429. 548. 216. 557. 534. 419. 149. 165. 275. 218.
 565. 105. 236. 506. 430. 380. 458. 489. 588. 305. 521. 251. 270. 282.
 369. 362. 113. 492. 549. 467. 404. 520. 566. 259. 244. 354. 502. 438.
 449. 231. 138. 457. 347. 359. 127. 474. 266. 531. 479. 276. 160. 590.
 198. 167. 432. 575. 196. 128. 169. 422. 495. 522. 320. 246. 343. 477.
 551. 237. 366. 1

In [28]:
for i in ['dist1', 'dist2']:
    print(i)
    print(train_transaction[i].unique(), train_transaction[i].nunique())

dist1
[ 852.   10.   nan ... 3177. 2175. 1736.] 2570
dist2
[  nan 1837.    6. ...  624. 2172. 1625.] 1648


In [29]:
for i in ['P_emaildomain', 'R_emaildomain']:
    print(i)
    print(train_transaction[i].unique(), train_transaction[i].nunique())

P_emaildomain
['yahoo.com' 'hotmail.com' 'frontier.com' 'gmail.com' 'msn.com' nan
 'anonymous.com' 'att.net' 'comcast.net' 'hotmail.co.uk' 'aol.com'
 'netzero.net' 'sbcglobal.net' 'icloud.com' 'centurylink.net' 'ymail.com'
 'verizon.net' 'live.com' 'netzero.com' 'q.com' 'outlook.com'
 'live.com.mx' 'cox.net' 'yahoo.com.mx' 'aim.com' 'bellsouth.net'
 'ptd.net' 'hotmail.fr' 'me.com' 'rocketmail.com' 'earthlink.net'
 'optonline.net' 'suddenlink.net' 'twc.com' 'cfl.rr.com' 'gmx.de'
 'roadrunner.com' 'windstream.net' 'web.de' 'mail.com' 'charter.net'
 'hotmail.de' 'outlook.es' 'frontiernet.net' 'gmail' 'mac.com'
 'hotmail.es' 'sc.rr.com' 'yahoo.fr' 'prodigy.net.mx' 'live.fr' 'juno.com'
 'yahoo.es' 'embarqmail.com' 'protonmail.com' 'cableone.net'
 'servicios-ta.com' 'yahoo.de' 'yahoo.co.uk' 'yahoo.co.jp'] 59
R_emaildomain
[nan 'gmail.com' 'hotmail.com' 'outlook.com' 'aol.com' 'hotmail.co.uk'
 'yahoo.com' 'optonline.net' 'comcast.net' 'anonymous.com' 'earthlink.net'
 'live.com' 'q.com' 'live.

In [30]:
for i in ['addr1', 'addr2']:
    print(i)
    print(train_transaction[i].unique(), train_transaction[i].nunique())

addr1
[204. 325. 472.  nan 264. 251. 231. 191. 110. 476. 203. 387. 315. 269.
 181. 337. 123. 512. 143. 498. 184. 330. 441. 485. 272. 226. 299. 384.
 420. 433. 494. 126. 148. 253. 310. 444. 177. 436. 324. 492. 327. 469.
 225. 511. 158. 220. 170. 448. 205. 356. 284. 418. 122. 163. 428. 343.
 502. 465. 308. 130. 296. 194. 302. 393. 431. 410. 404. 432. 254. 102.
 208. 275. 366. 167. 332. 161. 348. 258. 157. 365. 536. 171. 117. 374.
 244. 104. 386. 540. 297. 242. 218. 159. 295. 478. 341. 375. 369. 520.
 174. 303. 239. 443. 508. 430. 391. 396. 154. 189. 453. 335. 305. 477.
 215. 504. 276. 360. 429. 216. 499. 250. 459. 321. 248. 483. 155. 106.
 445. 376. 217. 409. 183. 227. 339. 166. 377. 385. 277. 458. 426. 359.
 145. 100. 349. 529. 134. 411. 486. 479. 247. 491. 439. 283. 152. 206.
 294. 221. 211. 236. 518. 142. 137. 112. 105. 435. 513. 408. 210. 199.
 474. 113. 260. 468. 279. 307. 268. 416. 516. 190. 259. 528. 128. 522.
 452. 151. 274. 270. 265. 526. 101. 399. 371. 467. 326. 521. 451. 314.


In [31]:
for i in ['M{}'.format(x) for x in range(1, 10)]:
    print(i)
    print(train_transaction[i].unique(), train_transaction[i].nunique())

M1
['T' nan 'F'] 2
M2
['F' 'T' nan] 2
M3
['F' 'T' nan] 2
M4
[nan 'M0' 'M1' 'M2'] 3
M5
[nan 'F' 'T'] 2
M6
['T' nan 'F'] 2
M7
[nan 'F' 'T'] 2
M8
[nan 'F' 'T'] 2
M9
[nan 'T' 'F'] 2
